In [1]:
from tensorflow.keras.models import load_model

# Load the model from a file
model = load_model('my_model.h5')

In [2]:
class Rectangle:
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
    def __str__(self):
        return f"Rectangle(x={self.x}, y={self.y}, width={self.width}, height={self.height})"
        

rectangle_list = []

x = 490
y = 988
width = 1007 - 490
height = 1060 - 990

for i in range(4):
    rec = Rectangle(x,y + (76 * i), width, height)
    rectangle_list.append(rec)

x = 1019
for i in range(4):
    rec = Rectangle(x,y + (76 * i), width, height)
    rectangle_list.append(rec)

In [3]:
import pyautogui
import os
import time

def timer(seconds):
    # Start a 5-second timer
    for i in range(seconds, 0, -1):
        print(f"Remaining: {i} seconds")
        time.sleep(1)

def screenshot(id):
    # Specify the directory where you want to save the screenshot
    directory = r"C:\Users\Eduardo\Pictures\Screenshots\persona3Telos"

    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    # Capture the screenshot
    screenshot = pyautogui.screenshot()

    # Save the screenshot to the specified directory
    screenshot.save(os.path.join(directory, f"{id}.png"))

import pydirectinput
import time

def change():
    # Press the 'l' key
    pydirectinput.press('l')
    #print('l key pressed')

    # Wait for 0.1 seconds
    time.sleep(0.1)

    # Press the 'm' key
    pydirectinput.press('m')
    #print('m key pressed')
    time.sleep(0.3)

In [4]:
import pyautogui
import numpy as np
import pickle
from PIL import Image

# Load the LabelEncoder from a file
with open('label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

def get_images(number):

    images = []

    #test
    #img_path = r"C:\Users\Eduardo\Pictures\Screenshots\persona3Telos\3283.png"

    #img = Image.open(img_path)
    
    for rectangle in rectangle_list:
        # Take a screenshot of the rectangle
        cropped_img = pyautogui.screenshot(region=(rectangle.x, rectangle.y, rectangle.width, rectangle.height))
        
        #test
        #cropped_img = img.crop((rectangle.x, rectangle.y, rectangle.x + rectangle.width, rectangle.y + rectangle.height))

         # Convert the image to RGB
        rgb_img = cropped_img.convert('RGB')

        hex_color = '65dfcd'

        # Convert the target hex color to RGB
        target_r, target_g, target_b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        target_color = np.array([target_r, target_g, target_b])

        # Convert the image to a NumPy array
        img_array = np.array(rgb_img)

        # Calculate the Euclidean distance between each pixel's color and the target color
        color_distance = np.linalg.norm(img_array - target_color, axis=2)

        # Create a mask for pixels that are similar to the target color
        mask = color_distance < 65

        # Change the color of the pixels based on the mask
        img_array[mask] = [0, 0, 0]  # black
        img_array[~mask] = [255, 255, 255]  # white

        # Convert the NumPy array back to an image
        rgb_img = Image.fromarray(img_array)

        # Convert the image to grayscale
        gray_img = rgb_img.convert('L')

        # Apply a threshold to make it binary
        binary_img = gray_img.point(lambda x: 0 if x<128 else 255, '1')

        img_array = np.array(binary_img)
        
        images.append(img_array)
    
    # Convert the list of images to a numpy array
    images = np.array(images)

    # Use the model to make predictions
    predictions = model.predict(images)

    # Decode the predictions to their original form
    decoded_predictions = le.inverse_transform([np.argmax(prediction) for prediction in predictions])

    #print decoded predicitons
    #print(decoded_predictions)
    #decoded_predictions.sort()
    return decoded_predictions



In [5]:
import logging

list_of_skills = [
    'High Counter',
    'Unshaken Will',
    'Panta Rhei',
    'Wind Boost',
    'Wind Amp',
    'Mind Charge',
    'Spell Master',
    'Salvation',
    ]
list_of_skills.sort()

mandatory_skills = [
    'High Counter',
    'Unshaken Will',
    'Panta Rhei',
    ]

mandatory_skills_set = set(mandatory_skills)

optional_skills = [
    'Wind Boost',
    'Wind Amp',
    'Spell Master',
    'Mind Charge',
    'Salvation',
]

optional_skills_set = set(optional_skills)

#test 
''' 
list_of_skills = [
    'High Counter',
    'Unshaken Will',
    'Panta Rhei',
    'Wind Boost',
    'Wind Amp',
    'Auto-Sukukaja',
    'Ghastly Wail',
    '1hSwd Master',
    ]
list_of_skills.sort()

'''

# Create a logger
logging.basicConfig(filename='predictions.log', level=logging.INFO)

skills_aquired = []
for i in range(6):
    if i == 2 or i == 3:
        skills_aquired.append(True)
    else:
        skills_aquired.append(False)

dictonary_number_key = { 
    1 : 'f6',
    2 : 'f7',
    3 : 'f8',
    4 : 'f9',
    5 : 'f10',
}

def verify_skills(predictions):
    # Convert both lists to sets
    predictions_set = set(predictions)

    if mandatory_skills_set.issubset(predictions_set):
        # pres F5 key
        logging.info('mandatory set found')
        if not skills_aquired[0]:
            pydirectinput.keyDown('f5')
            pydirectinput.keyUp('f5')
            #print('mandatory set found')
            skills_aquired[0] = True

        n = len(optional_skills_set & predictions_set)
        if len(optional_skills_set & predictions_set) >= 1:
            logging.info(f'{n} optional skills found')
            if not skills_aquired[n]:
                pydirectinput.keyDown(dictonary_number_key[n])
                pydirectinput.keyUp(dictonary_number_key[n])
                #print(f'{n} optional skills found')
                skills_aquired[n] = True

                if n == 5:
                    return True
    return False

In [6]:
import pygetwindow as gw


def check_window_change(active_window_title):
    # Check if the active window has changed
    if gw.getActiveWindow().title != active_window_title:
        print('Active window has changed. Stopping the program.')
        return True
    return False




timer(10)
# Press the F4 key
#pydirectinput.keyDown('f4')
#pydirectinput.keyUp('f4')
# Get the title of the currently active window
active_window_title = gw.getActiveWindow().title
while True:
    if check_window_change(active_window_title):
        break
    predictions = get_images(i)
    # Log the predictions vector
    logging.info(f'{predictions}')
    if verify_skills(predictions):
        print('found')
        pydirectinput.keyDown('esc')
        pydirectinput.keyUp('esc')
        print('esc key pressed')
        break
    if check_window_change(active_window_title):
        break
    change()

Remaining: 10 seconds
Remaining: 9 seconds
Remaining: 8 seconds
Remaining: 7 seconds
Remaining: 6 seconds
Remaining: 5 seconds
Remaining: 4 seconds
Remaining: 3 seconds
Remaining: 2 seconds
Remaining: 1 seconds
1/1 [==============================] - 0s 129ms/step


AttributeError: 'NoneType' object has no attribute 'title'